# load packages + define function

In [233]:
using Statistics
using DelimitedFiles
using NumericalIntegration
using Plots
plotlyjs()
theme(:juno)

In [1]:
function hist(array,min,max,Δ)
    #use it like this: hist(rand(1:10,10^6),0.5,10.5,1)
    try
        Integer((max-min)/Δ)
    catch
        println("(max-min)/Δ is not an Integer!")
    end
    y=zeros(Integer((max-min)/Δ))
    error=0
    for el in array
        if min<el<=max
            y[Integer(ceil((el-min)/Δ))]+=1
        else
            if error == 0
                error=1
                println("Range insufficient. Element with value $(el) was found and ignored.")
            end
        end
    end
    x=Array(min:Δ:max)
    return [0.5*(x[1:end-1]+x[2:end]), y]
end

hist (generic function with 1 method)

# load downloaded data + translate into array + save to file
Downloaded from https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page \
I'm only using the data from "Yellow Taxi".

In [174]:
for month in 1:12
    #load data
    if month<10
        data=readdlm("../../RawData/TaxiNYC/yellow_tripdata_2019-0$(month).csv")
    else
        data=readdlm("../../RawData/TaxiNYC/yellow_tripdata_2019-$(month).csv")
    end

    #get distances
    distances=zeros(length(data[2:end,3]))
    for jj in 1:length(data[2:end,3])
        str=data[1+jj,3]
        mys=""
        count=0
        for ind in 1:length(str)
            sym=str[ind]
            if sym==','
                count+=1
                continue
            end
            if count==2
                mys*=sym
            elseif count == 3
                break
            end
        end
        distances[jj]=parse(Float64,mys)
    end

    #get start times
    timeStart=zeros(length(data[2:end,2]))
    timeEnd=zeros(length(data[2:end,2]))
    for jj in 1:length(data[2:end,2])
        str=data[1+jj,2]
        timeStart[jj]=parse(Float64,str[1:2])*60+parse(Float64,str[4:5])
        str=data[1+jj,3]
        timeEnd[jj]=parse(Float64,str[1:2])*60+parse(Float64,str[4:5])
    end

    #save to variables
    if month<10
        myCode="d0$(month)=distances;t10$(month)=timeStart;t20$(month)=timeEnd"
    else
        myCode="d$(month)=distances;t1$(month)=timeStart;t2$(month)=timeEnd"
    end
    eval(Meta.parse(myCode))

    #proudly announce progress
    println("month $(month) done.")
end

month 1 done.
month 2 done.
month 3 done.
month 4 done.
month 5 done.
month 6 done.
month 7 done.
month 8 done.
month 9 done.
month 10 done.
month 11 done.
month 12 done.


#### write distances to files

In [3]:
for month in 1:12
    if month<10
        myCode="writedlm(\"distances/d0$(month)\",d0$(month))"
    else
        myCode="writedlm(\"distances/d$(month)\",d$(month))"
    end
    eval(Meta.parse(myCode))
end

LoadError: UndefVarError: d01 not defined

#### write times to files

In [177]:
for month in 1:12
    if month<10
        myCode="writedlm(\"times/t10$(month)\",t10$(month));writedlm(\"times/t20$(month)\",t20$(month))"
    else
        myCode="writedlm(\"times/t1$(month)\",t1$(month));writedlm(\"times/t2$(month)\",t2$(month))"
    end
    eval(Meta.parse(myCode))
end

# distances

#### read distances from files

In [164]:
for month in 1:12
    if month<10
        myCode="d0$(month)=readdlm(\"distances/d0$(month)\")"
    else   
        myCode="d$(month)=readdlm(\"distances/d$(month)\")"     
    end
    eval(Meta.parse(myCode))
end

#### combine distances into one array

In [165]:
distances=vcat(d01,d02,d03,d04,d05,d06,d07,d08,d09,d10,d11,d12);

#### histogram

In [237]:
dataset=distances
min=0
max=50
Δ=0.1
his=hist(dataset,min,max,Δ)
println("$(round(100*sum(his[2])/length(dataset);digits=2))% of data used.")
Z=integrate(his[1], his[2])
his[2]./=Z;

Range insufficient. Element with value 0.0 was found and ignored.
98.94% of data used.


In [261]:
plot(his[1],his[2],xlim=[0,6],marker=:hex,xlab="distance / km",ylab="frequency",title="taxi data from NYC, year 2019")

<!DOCTYPE html>
 
 
 Plots.jl

In [343]:
savefig("traveldistancesHistogramNYC2019.html")

#### remaining passengers

In [265]:
Γ=zeros(length(his[2]));
his[2][1]=0
temp=his[2]./his[1]
for z in 1:length(Γ)
    Γ[z]=integrate(his[1][z:end],temp[z:end])
end
F=cumul_integrate(his[1],-Γ).+1;

In [276]:
plot(his[1],F,xlim=[0,10],marker=:hex,xlab="distance / km",ylab="fraction of remaining passengers",title="taxi data from NYC, year 2019")

<!DOCTYPE html>
 
 
 Plots.jl

#### analytic approximation

In [170]:
D=mean(distances)
k=3.1
x=Array(0.01:0.01:20)
y=exp.(-(x./D).^-1).*((x./D).^-k)
y./=sum(y)*0.01
plot!(x,y,xlim=[0,12],xlab="distance / km",ylab="frequency",title="taxi data from NYC, year 2019",xticks=1:50)

<!DOCTYPE html>
 
 
 Plots.jl

#### integral distribution
..but *-1 +1

In [273]:
plot(his[1],cumul_integrate(his[1],-his[2]).+1,marker=:hex,xlim=[0,12],xlab="distance / km",ylab="remaining customers",title="taxi data from NYC, year 2019",xticks=0:2:100,yticks=0:0.2:1)

<!DOCTYPE html>
 
 
 Plots.jl

#### log-log histogram

In [172]:
#log-log
logd1=log.(10,his[1])
logd2=log.(10,his[2])
plot(logd1,logd2,marker=:hex,xlab="log_10 distance / km",ylab="log_10 frequency",title="taxi data from NYC, year 2019")

<!DOCTYPE html>
 
 
 Plots.jl

#### power law exponent - rough estimate

In [345]:
right=300
left=50
(logd2[right]-logd2[left])/(logd1[right]-logd1[left])

-3.1900103156234354

# travel times

#### read times from files

In [178]:
for month in 1:12
    if month<10
        myCode="t10$(month)=readdlm(\"times/t10$(month)\");t20$(month)=readdlm(\"times/t20$(month)\")"
    else   
        myCode="t1$(month)=readdlm(\"times/t1$(month)\");t2$(month)=readdlm(\"times/t2$(month)\")"
    end
    eval(Meta.parse(myCode))
end

#### combine times into one array

In [179]:
t1=vcat(t101,t102,t103,t104,t105,t106,t107,t108,t109,t110,t111,t112);

In [180]:
t2=vcat(t201,t202,t203,t204,t205,t206,t207,t208,t209,t210,t211,t212);

#### histogram

In [186]:
dataset=t1
min=0
max=1439
Δ=1
his=hist(dataset,min,max,Δ)
println("$(round(100*sum(his[2])/length(dataset);digits=2))% of data used.")

Range insufficient. Element with value 0.0 was found and ignored.
99.94% of data used.


#### start times

In [187]:
plot(his[1],his[2]/365,marker=:hex,xlab="distance / km",ylab="requests per minute",title="taxi data from NYC, year 2019",xticks=0:120:1440)

<!DOCTYPE html>
 
 
 Plots.jl

In [161]:
savefig("startTimesHistogramNYC2019.html")

#### travel times

In [184]:
plot(his[1],his[2],marker=:hex,xlab="distance / km",ylab="frequency",title="taxi data from NYC, year 2019")

<!DOCTYPE html>
 
 
 Plots.jl